### Scrape Location Data for Family Dollar in U.S.

In [1]:
import requests
from bs4 import BeautifulSoup
import json
from pandas import DataFrame as df

In [2]:
page = requests.get("https://minimalpairs.net/en/fr")
soup = BeautifulSoup(page.text, 'html.parser')

In [69]:
# find all 
rows = soup.find_all('tr')

In [93]:
pairs = []

for row in rows:
    ipa_cells = row.find_all('td', class_ = 'ipa')
    is_ipa = len(ipa_cells) > 0
    if is_ipa:
        wordpair_spans = row.find_all('span', class_ = 'wordpair')

        pairs.append({
            "characters": [ipa_cells[0].contents[0], ipa_cells[1].contents[0]],
            "words": list(map(lambda wordpair_span: [{
                "word": wordpair_span.contents[0].strip(),
                "ipa": wordpair_span.contents[1].contents[0]
            },{
                "word": wordpair_span.contents[2].replace('|', '').strip(),
                "ipa": wordpair_span.contents[3].contents[0]
            }], wordpair_spans))
        })

pairs

[{'characters': ['p', 'b'],
  'words': [[{'word': 'blanc', 'ipa': '/blɑ̃/'},
    {'word': 'plan', 'ipa': '/plɑ̃/'}],
   [{'word': 'p', 'ipa': '/pe/'}, {'word': 'b', 'ipa': '/be/'}],
   [{'word': 'pain', 'ipa': '/pɛ̃/'}, {'word': 'bain', 'ipa': '/bɛ̃/'}],
   [{'word': 'par', 'ipa': '/paʁ/'}, {'word': 'bar', 'ipa': '/baʁ/'}],
   [{'word': 'pas', 'ipa': '/pa/'}, {'word': 'bas', 'ipa': '/ba/'}],
   [{'word': 'peau', 'ipa': '/po/'}, {'word': 'beau', 'ipa': '/bo/'}],
   [{'word': 'pis', 'ipa': '/pi/'}, {'word': 'by', 'ipa': '/bi/'}],
   [{'word': 'pont', 'ipa': '/pɔ̃/'}, {'word': 'bon', 'ipa': '/bɔ̃/'}],
   [{'word': 'port', 'ipa': '/pɔʁ/'}, {'word': 'bord', 'ipa': '/bɔʁ/'}],
   [{'word': 'puce', 'ipa': '/pys/'}, {'word': 'bus', 'ipa': '/bys/'}]]},
 {'characters': ['p', 't'],
  'words': [[{'word': 'pont', 'ipa': '/pɔ̃/'},
    {'word': 'ton', 'ipa': '/tɔ̃/'}],
   [{'word': 'prix', 'ipa': '/pʁi/'}, {'word': 'tri', 'ipa': '/tʁi/'}],
   [{'word': 't', 'ipa': '/te/'}, {'word': 'p', 'ipa': '/pe/'}